# Machine Learning Algorithms

## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris

## General Purpose Methods

### Distance

In [2]:
class Distance:
    @staticmethod
    def squared_euclidean_distance(X,y):
        # Tile the vector y to create a matrix with the same number 
        # of rows as X and the same number of columns as y
        y = np.tile(y, (X.shape[0], 1))
        
        # Calculate the squared Euclidean distance between each row of x and y
        squared_distance = np.square(X - y).sum(axis=1)
        
        return squared_distance

    @staticmethod
    def euclidean_distance(X, y):
        squared_distance = Distance.squared_euclidean_distance(X,y)
        
        # Take the square root to get the Euclidean distance
        distance = np.sqrt(squared_distance)
    
        return distance

## KNN Classification

### Load Zoo dataset

In [3]:
column_names = ["name", "hair", "feathers", "eggs", "milk", "airbone", "aquatic", "predator", "toothed", "backbone", "breathes", "venomous", "fins", "legs", "tail", "domestic", "catsize", "type"]
column_types = {"name" : str,
                "hair" : int,
                "feathers" : int,
                "eggs" : int,
                "milk" : int,
                "airbone" : int,
                "aquatic" : int,
                "predator" : int,
                "breathes" : int,
                "venomous" : int,
                "fins" : int,
                "legs" : int,
                "tail" : int,
                "domestic" : int,
                "catsize" : int,
                "type" : int
                }

zoo_df = pd.read_csv("../resources/zoo/zoo.data", delimiter=",", header=None, names=column_names, dtype=column_types)
zoo_df.head(-2)

,name,hair,feathers,eggs,milk,airbone,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,vole,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,0,1
95,vulture,0,1,1,0,1,0,1,0,1,1,0,0,2,1,0,1,2
96,wallaby,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,wasp,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6


In [4]:
# split initial data into train and test (50 examples for test)
X_train, X_test, y_train, y_test = train_test_split(zoo_df.drop(["name", "type"], axis=1).to_numpy(),
                                                    zoo_df["type"].to_numpy(),
                                                    test_size=20)

# split rest of the train data into train and dev
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, 
                                                  test_size=30)    

### Load Iris dataset

In [18]:
iris_data = load_iris(as_frame=True)
iris_data.frame

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [19]:
# split initial data into train and test (50 examples for test)
X_train, X_test, y_train, y_test = train_test_split(iris_data.data.to_numpy(), 
                                                    iris_data.target.to_numpy(), 
                                                    test_size=50)


# split rest of the train data into train and dev (50 examples for dev)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, 
                                                  test_size=50)

### Custom kNN Classifier

In [7]:
from statistics import mode

class kNN:
    def __init__(self, k):
        # k nearest neighbours hyperparameter
        self.k = k

        # X_train: n*m matrix (n examples, m features)
        # X[i] = [xi1, xi2, ..., xim] for i=1 to n
        self.X_train = None
        
        # y_train: n*1 vector (class of each example)
        # y[i] = class of X[i]  for i=1 to n
        self.y_train = None
    
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        Ntest = X_test.shape[0]
        predicted_classes = list()

        for test_example_idx in range(Ntest):
            # Calculate the distance for all train examples
            d = Distance.euclidean_distance(self.X_train, X_test[test_example_idx,:])

            # Return the indices of the K closest instances
            k_closest = np.argsort(d)[:self.k]

            # Find the classes of the k closest instances
            y = self.y_train[k_closest]

            # mode: returns the most frequent (majority vote)
            predicted_classes.append(mode(y))

        return np.array(predicted_classes)

In [8]:
def find_best_k(X_train, y_train, X_dev, y_dev):
  best_k = 0
  best_dev_accuracy = .0
  best_classifier = None 

  for k in tqdm(range(1, 11)):
    knn = kNN(k)  # knn object with current k
    knn.fit(X_train, y_train) # fit with current k in the training data

    predictions = knn.predict(X_dev) # predict with current k using the development data
    accuracy = accuracy_score(y_dev, predictions) # count accuracy

    if accuracy > best_dev_accuracy: 
      best_dev_accuracy = accuracy
      best_k = k
      best_classifier = knn
    
    print('Accuracy for k={0}: {1}'.format(k, accuracy))


  print('\nBest dev accuracy:', best_dev_accuracy)
  print('Best k:', best_k)

  # predict the test data using the best classifier
  test_preds = best_classifier.predict(X_test)
  print('Test accuracy:', accuracy_score(y_test, test_preds))
  print()

  return best_k

### Training & Testing

#### Custom kNN Classifier

In [9]:
best_k = find_best_k(X_train, y_train, X_dev, y_dev)

# Training kNN Classifier
knn_custom = kNN(best_k) 
knn_custom.fit(X_train, y_train)

# Using kNN Classifier
y_custom = knn_custom.predict(X_test)
print(classification_report(y_test, y_custom))

100%|██████████| 10/10 [00:00<00:00, 278.04it/s]

Accuracy for k=1: 0.98
Accuracy for k=2: 0.98
Accuracy for k=3: 0.96
Accuracy for k=4: 0.98
Accuracy for k=5: 0.92
Accuracy for k=6: 0.96
Accuracy for k=7: 0.94
Accuracy for k=8: 0.96
Accuracy for k=9: 0.96
Accuracy for k=10: 0.96

Best dev accuracy: 0.98
Best k: 1
Test accuracy: 0.96

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       0.94      0.94      0.94        17
           2       0.93      0.93      0.93        15

    accuracy                           0.96        50
   macro avg       0.96      0.96      0.96        50
weighted avg       0.96      0.96      0.96        50



#### Scikit-Learn kNN Classifier

In [10]:
# Training kNN Classifier
knn_scikit = KNeighborsClassifier(n_neighbors=best_k) 
knn_scikit.fit(X_train, y_train)

# Using kNN Classifier
y_scikit = knn_scikit.predict(X_test)
print(classification_report(y_test, y_scikit))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       0.94      0.94      0.94        17
           2       0.93      0.93      0.93        15

    accuracy                           0.96        50
   macro avg       0.96      0.96      0.96        50
weighted avg       0.96      0.96      0.96        50



## K-means Clustering

### Custom k-means

In [11]:
class kMeans:
    def __init__(self, k, max_iters):
        # k = number of clusters
        self.k = k
        # define the maximum number of iterations
        self.max_iters = max_iters
        # the centroids for all labels
        self.centroids = None

    @staticmethod
    def cost_function(X, c, labels): 
        return 0.5*sum(Distance.squared_euclidean_distance(X[i].reshape((1,X[i].shape[0])), c[labels[i]]) for i in range(X.shape[0]))
    
    def fit(self, X):
        # Choose k random points of data as centroids
        indices = np.random.choice(X.shape[0], self.k, replace=False)
        self.centroids = X[indices]

        costs = list()

        for iter in range(self.max_iters):
            # Expectation step
            labels = self.assign_label(X)  # Assign each data point to the nearest centroid
            costs.append(self.cost_function(X, self.centroids, labels).item()) # Calculate the cost 

            # Maximization step
            new_centroids = np.array([np.mean(X[labels == label], axis=0) for label in range(self.k)])
            costs.append(self.cost_function(X, new_centroids, labels).item()) # Calculate the new cost
            
            # If the algorithm has converged, then stop
            if np.all(self.centroids == new_centroids):
                break
                
            self.centroids = new_centroids

        return labels, costs

    def assign_label(self, X):
        # Assign each data point to the nearest centroid 
        distances = np.array([Distance.squared_euclidean_distance(self.centroids, x) for x in X])
        labels = np.argmin(distances, axis=1)

        return labels

In [12]:
kmeans = kMeans(3, 4)
labels, costs = kmeans.fit(X_train)

for cost in costs:
    print(cost)

21.475000000000012
12.393666666666663
12.393666666666663
12.393666666666663


## Principal Component Analysis

### With SVD & Classic PCA

In [13]:
class PCA:
    def __init__(self, m):
        self.m = m

    def svd_reduction(self, X):
        U, S, V = np.linalg.svd(X, full_matrices=False)
        print( "U", U.shape, "S", S.shape, "V", V.shape )
        mu = X.mean(axis=0)

        eigvecs = V[:self.m,:]
        eigvals = S[:self.m]
        print( 'X', X.shape, 'eigvecs', eigvecs.shape, 'eigvals', eigvals.shape )
        Z = (X-mu).dot(eigvecs.T)
        print( 'Z', Z.shape )
        return Z, eigvecs, eigvals, mu
    
    def classic_reduction(self, X):
        mu = X.mean(axis=0).reshape( (1,-1) )

        normalized_X = X-mu

        S = (1/X.shape[0]) * normalized_X.T.dot( normalized_X )

        eigvectors, eigvals = self.eigsort( S )

        U = eigvectors[:,:self.m]
        Lambdas = eigvals[:self.m]
        print( X.shape, U.shape, Lambdas.shape )

        Z = normalized_X.dot(U)
        return Z, U, Lambdas, mu
    
    def eigsort(self, A):
        eigvals, U = np.linalg.eig(A)
        # sort eigenvalues in descending order
        order = np.argsort(eigvals)[::-1]
        eigvals = eigvals[order]
        #re-arrange the eigenvectors
        U = U[:,order]
        return U, eigvals

In [14]:
m = 3

iris_data = load_iris(as_frame=True)
iris_data.frame

# split initial data into train and test (50 examples for test)
X_train, X_test, y_train, y_test = train_test_split(iris_data.data.to_numpy(), 
                                                    iris_data.target.to_numpy(), 
                                                    test_size=50)


# split rest of the train data into train and dev (50 examples for dev)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, 
                                                  test_size=50)

pca = PCA(m)
z, eva, eve, mu = pca.svd_reduction(X_train)
print(z.shape)

U (50, 4) S (4,) V (4, 4)
X (50, 4) eigvecs (3, 4) eigvals (3,)
Z (50, 3)
(50, 3)


In [15]:
z, u, eva, mu = pca.classic_reduction(X_train)
print(z.shape)

(50, 4) (4, 3) (3,)
(50, 3)


## Least Squares Classification

* From Bishop, page 184

Consider a classification problem with  classes, with a 1-of-K binary coding scheme for the target vector t. Each class  has its own linear model.

Each class  is described by its own linear model so that

Conviniently using vector notation,

where 
 is a matrix whose  column comprises the  -dimensional vector 
 and 
 is the corresponding augmented input vector  with a dummy input . Assign new input  to the class  if .

Consider a training data set  where  and define a matrix  whose  row is the vector 
.

The sum-of-squares error function can then be written as

 
Setting the derrivative wrt 
 to zero, and rearranging, we obtain the solution

We then obtain the discriminant functions as


In [66]:
class Least_Squares:

    def __init__(self):
        self.W = None

    def fit(self, X_train, y_test):
        # Insert 1 as the first column in X
        # augment the input vectors with a dummy input x_0 = 1
        X = np.insert(X_train, 0, 1, axis=1)
        """
        Y = [1,
             2,
             ..,
             0]
        T = [[0, 1,..., 0],
                ...
             [1, 0,..., 0]]
        """
        T = pd.get_dummies(y_test).astype(int)
        # Compute the pseudo-inverse
        self.W = np.linalg.pinv(X).dot(T)
        print(self.W.shape)

    def predict(self, X_test):
        X = np.insert(X_test, 0, 1, axis=1)
        Y = self.W.T.dot(X.T)
        print(Y)
        classes = np.argmax(Y, axis=0)
        return classes

In [67]:
ls = Least_Squares()

ls.fit(X_train, y_train)
predictions = ls.predict(X_test=X_test)
print(predictions)

(5, 3)
[[ 0.9194588   0.90537538  0.19248004  0.84461602 -0.04907994  0.87116528
   0.20137677  0.17397173  0.16642496  0.22811154 -0.03565726 -0.0901902
  -0.03109505  0.82378831 -0.06220653  0.1021798   1.13542276  0.87925953
   0.88461911 -0.02191884  0.88037264  0.15047332 -0.05468898 -0.0495978
   1.12660853  0.00310204  0.14186672  0.2475289   0.93479152  0.83075839
   0.04089152  0.9863139   0.21400957  0.83307819  0.86596753  0.88749706
  -0.16795121  0.04019009  0.80957977 -0.00600981  0.0985708   0.23457674
   0.12792618  0.20763435 -0.16622667  0.98603482 -0.09620898  0.14489543
   0.02601269  0.89129156]
 [ 0.05697326  0.23168929  0.41960208  0.21025676  0.14290142  0.2149719
   0.27609754  0.31394546  0.4132005   0.24158827  0.15699111  0.1082563
   0.44250318  0.30011737  0.15227597  0.45804497 -0.10799062  0.25557782
   0.1710682   0.52879534  0.24327066  0.30004633  0.24527964  0.03074373
  -0.01519739  0.22518485  0.55315595  0.44622925  0.18074816  0.25326007
   0.224

In [68]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.35      0.52        17
           2       0.56      1.00      0.72        14

    accuracy                           0.78        50
   macro avg       0.85      0.78      0.75        50
weighted avg       0.88      0.78      0.76        50

